# Bit Reader
Encode/Decode a set of bits

Initializes with parameter `options`, which must be a dictionary with the following format:

- keys must be a str with the bits places, example: '0-1' means bit 0 and bit 1

- values must be a dictionary with the bit value as the key and the category (str) as value. Categories must be unique.

- Encode: given a category/categories return a list of possible values
- Decode: given a value return a list of categories

## Example

MOD09 (http://modis-sr.ltdri.org/guide/MOD09_UserGuide_v1_3.pdf)
(page 28, state1km, 16 bits):

In [ ]:
import ee
ee.Initialize()

In [ ]:
from geetools import bitreader, cloud_mask

In [ ]:
options = {
 '0-1': {0:'clear', 1:'cloud', 2:'mix'},  # cloud state
 '2-2': {0: 'no_shadow', 1:'shadow'},  # cloud shadow (bit 0 is not needed)
 '6-7': {0:'climatology', 1:'low', 2:'average', 3:'high'}  # land/water flag
 }

In [ ]:
reader = bitreader.BitReader(options, 16)

Internally it computes a dict with
- bit_length (length of the group of bits)
- lshift (left shift)
- shifted (shifted places)

In [ ]:
reader.info

In [ ]:
print('bit length', reader.bit_length)

DECODE ONE VALUE

In [ ]:
value = 204
bits = reader.getBin(value)
print('204:', bits)

In [ ]:
reader.decode(204)

MATCH ONE VALUE

In [ ]:
reader.match(204, 'cloud')

In [ ]:
reader.match(204, 'shadow')

ENCODE A VALUE (EXCLUSIVELY)

In this case, shadow is 00000100 (4) and **not** 00000101 (5)

In [ ]:
reader.encode('shadow')

In [ ]:
reader.encode('clear')

In [ ]:
reader.encode('no_shadow')

ENCODE A VALUE (ALL)

This will get **all** values (all combinations where the bit is set)

In [ ]:
print(reader.encodeOne('shadow')[0:100])

In [ ]:
print(reader.encodeOne('cloud')[0:100])

ENCODE AND

In [ ]:
print(reader.encodeAnd('cloud', 'shadow')[0:100])

### DECODE AN IMAGE

In [ ]:
import ee

In [ ]:
import ipygee as ui

In [ ]:
Map = ui.Map()
Map.show()

In [ ]:
modcol = ee.ImageCollection('MODIS/006/MOD09GA').sort('system:time_start', False)

In [ ]:
mod = ee.Image(modcol.first())

BANDS

In [ ]:
red = 'sur_refl_b01'
green = 'sur_refl_b04'
blue = 'sur_refl_b03'

In [ ]:
qa = 'state_1km'

In [ ]:
qa_mask = mod.select(qa)

In [ ]:
Map.addLayer(mod, {'bands':[red, green, blue], 'min':0, 'max':5000}, 'Original')

In [ ]:
Map.addLayer(qa_mask, {'min':0, 'max':reader.max}, 'QA')

APPLY THE `BitReader` TO THE BAND THAT HOLDS THE BIT INFORMATION

In [ ]:
mask = reader.decodeImage(mod, qa)

In [ ]:
Map.addLayer(mask.select(['cloud']), {'min':0, 'max':1}, 'Clouds')

`BitReader` INFORMATION FOR KNOW COLLECTIONS AVAILABLE IN `geetools.cloud_mask` MODULE

In [ ]:
from geetools import cloud_mask

In [ ]:
state1km = cloud_mask.BITS_MODIS09GA

In [ ]:
state1km